In this notebook, I will create a pipeline mechanism to feed the query data for the model to predict the results. This is an important aspect of modeling to check how the model does prediction on a single query point.

From `03-Modeling-FI.ipynb` notebook, I noticed __Gradient Boosting__ ensemble classifier outperformed all the models including Random Forest and XGBoost classifiers. Though train loss of Random Forest and XGBoost classifiers is negligible, the cross-validation loss is more, which substantiates the fact that both the models are overfitting.

__1. Packages__

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.display import display

In [3]:
from matplotlib import pyplot as plt
from matplotlib import style
style.use(style='seaborn-deep')

In [4]:
from tabulate import tabulate

In [5]:
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns

In [6]:
from sklearn.preprocessing import MinMaxScaler

__2. Features and target__

In [7]:
features = ['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']
target = 'class'

__3. Fetch the raw data__

In [8]:
def fetch_data(features):
    """
    This function fetches the raw data.
    """
    data = {f: [float(input("  '{}': ".format(f)))] for f in features}
    df = pd.DataFrame(data=data)
    print("Raw data is fetched successfully.")
    return df

In [9]:
df = fetch_data(features=features)

  'alpha': 33
  'delta': 23
  'u': 12
  'g': 34
  'r': 22
  'i': 9
  'z': 10
  'redshift': .1
Raw data is fetched successfully.


__4. Preprocess the raw data__

In [10]:
def preprocess(df, features):
    """
    This function preprocess the rae data.
    """
    with open(file='scaling.pkl', mode='rb') as pre_pkl:
        scaling = pickle.load(file=pre_pkl)
    
    df = scaling.transform(X=df)
    df = pd.DataFrame(data=df, columns=features)
    return df

In [11]:
df = preprocess(df=df, features=features)
display(df)

,alpha,delta,u,g,r,i,z,redshift
0,0.091653,0.403791,0.046076,1.168843,0.616611,-0.020727,0.019607,0.015663


__5. Feature engineering on preprocessed data__

In [12]:
def featurize(df):
    """
    This function featurizes the dataframe.
    It selects the important features obtained using RF.
    Please refer 02-Modeling and 03-Modeling-FI notebooks.
    """
    fi_cols = ['g', 'redshift', 'g-r', 'i-z', 'u-r', 'i-r', 'z-r']
    df['g-r'] = df['g'] - df['r']
    df['i-z'] = df['i'] - df['z']
    df['u-r'] = df['u'] - df['r']
    df['i-r'] = df['i'] - df['r']
    df['z-r'] = df['z'] - df['r']
    df = df[fi_cols]
    return df

In [13]:
df = featurize(df=df)
display(df)

,g,redshift,g-r,i-z,u-r,i-r,z-r
0,1.168843,0.015663,0.552232,-0.040334,-0.570535,-0.637337,-0.597003


__6. Predictions__

In [14]:
def prediction(X):
    """
    This functions predicts the datapoint.
    """
    with open(file='gb_classifier.pkl', mode='rb') as m_pkl:
        clf, sig_clf = pickle.load(file=m_pkl)
    
    pred = sig_clf.predict(X=X)
    print("Predicted class: {}".format(pred))

In [15]:
prediction(X=df)

Predicted class: ['GALAXY']


__7. Machine learning pipeline__

In [16]:
def ml_pipeline(features):
    """
    This is a local machine learning application.
    """
    print("Please provide the data for below features.")
    df = fetch_data(features=features)
    df = preprocess(df=df, features=features)
    df = featurize(df=df)
    prediction(X=df)

In [17]:
ml_pipeline(features=features)

Please provide the data for below features.
  'alpha': 1
  'delta': 1
  'u': 1
  'g': 1
  'r': 1
  'i': 1
  'z': 1
  'redshift': 1
Raw data is fetched successfully.
Predicted class: ['QSO']
